# 02.03 Geospatial Analysis: Neighborhood Council & Census Block Layered Visualizations
## Issue 1279
[Census Blocks Shapefile Source - 2020 US Census Blocks ](https://www2.census.gov/geo/tiger/TIGER2020PL/STATE/06_CALIFORNIA/06037/)
 | [NC Shapefile Source - City of LA](https://data.lacity.org/City-Infrastructure-Service-Requests/Neighborhood-Councils-Certified-/fu65-dz2f)
 
 ---

# Package & Data Imports
---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# https://stackoverflow.com/questions/13440102/getting-bounding-box-of-city
import geopandas as gpd

# https://www.analyticsvidhya.com/blog/2020/06/guide-geospatial-analysis-folium-python/
import folium
from branca.element import Figure
from folium.plugins import TimeSliderChoropleth

%matplotlib inline
import warnings
warnings.simplefilter("ignore")
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# 311 Data 10/01/2021-10/01/2022 from API see 01 notebook
raw_df = pd.read_csv('../data/clean_01Oct21_01Oct22_api.csv')
df = raw_df.copy()

# https://geopandas.org/en/stable/getting_started/introduction.html
# NC boundaries: https://data.lacity.org/City-Infrastructure-Service-Requests/Neighborhood-Councils-Certified-/fu65-dz2f
raw_gdf_nc = gpd.read_file('../data/Neighborhood Councils (Certified)/geo_export_88bb18d9-f96c-4351-8be9-594f258ed0d3.shp')
gdf_nc = raw_gdf_nc.copy()

# Block boundaries: https://www2.census.gov/geo/tiger/TIGER2020PL/STATE/06_CALIFORNIA/06037/
# tl_2020_06037_tabblock20.zip
raw_gdf_blk = gpd.read_file("../data/tl_2020_06037_tabblock20/tl_2020_06037_tabblock20.shp")
gdf_blk = raw_gdf_blk.copy()

# merged nc and blk geodata
gdf_blk_nc_raw = gpd.read_file('../data/SHAPE-clean_01Oct21_01Oct22_nc_blk/clean_01Oct21_01Oct22_nc_blk.shp')
gdf_blk_nc = gdf_blk_nc_raw.copy()

# merged request dataset with nc block geodata
df_blk_nc_raw = gpd.read_file('../data/SHAPE-clean_01Oct21_01Oct22_nc_blk_req/clean_01Oct21_01Oct22_nc_blk_req.shp')
df_blk_nc = df_blk_nc_raw.copy()

# Neighborhood Council and Block-by-Block Layered Map
---

In [ ]:
# create groupby neighborhood council
df_nc_ct = df.groupby('councilName')['councilId'].value_counts().reset_index(name = 'count')
print(df_nc_ct.shape)
df_nc_ct.head()

In [ ]:
# merge NC grouping with geometries
gdf_nc_plt = pd.merge(gdf_nc, df_nc_ct, how = 'left', left_on = 'objectid', right_on = 'councilId')
gdf_nc_plt.drop(columns = 'objectid', inplace = True)

# incase need to minimize dataset for testing
# gdf_nc_plt = gdf_nc_plt.sample(frac = .01, random_state = 42)

# sort and remove unnecessary columns
gdf_nc_plt = gdf_nc_plt.sort_values(by = 'councilId').reset_index(drop = True)
gdf_nc_plt = gdf_nc_plt[['geometry', 'councilId', 'councilName', 'count']]

# put geometry in format for folium
gdf_nc_plt = gdf_nc_plt.to_crs(epsg=4326)

# check geodataframe
print(gdf_nc_plt.shape)
gdf_nc_plt.head()

In [ ]:
# BLOCK
# create groupby neighborhood council
df_blk_ct = df_blk_nc.groupby(['councilId','councilNam'])['GEOID20'].value_counts().reset_index(name = 'count')
print(df_blk_ct.shape)
df_blk_ct.head()

In [ ]:
# merge block grouping with geometries
gdf_blk_plt = pd.merge(gdf_blk_nc, df_blk_ct, how = 'inner', on = 'GEOID20')

# incase need to minimize dataset for testing
# gdf_blk_plt = gdf_blk_plt.sample(frac = .01, random_state = 42)

# sort and remove unnecessary columns
gdf_blk_plt = gdf_blk_plt.sort_values(by = 'GEOID20').reset_index(drop = True)
gdf_blk_plt = gdf_blk_plt[['GEOID20', 'geometry', 'councilId_x', 'name', 'count']]
gdf_blk_plt.rename(columns = {'councilId_x':'councilId', 'name_x':'councilName'}, inplace = True)

# check geodataframe
print(gdf_blk_plt.shape)
gdf_blk_plt.head()

In [ ]:
# https://geopandas.org/en/stable/gallery/polygon_plotting_with_folium.html

# basemap
fig = Figure(height = 600, width = 1000)
map = folium.Map(location = (34.03, -118.39), tiles='cartodbpositron', zoom_start=9.5)
fig.add_child(map)

# neighborhood council requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_nc = folium.Choropleth(
    geo_data = gdf_nc_plt.to_json(),
    name="NC choropleth",
    data = gdf_nc_plt,
    columns=['councilName', 'count'],
    key_on="feature.properties.councilName",
    fill_color="YlGn",
    fill_opacity=0.7,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    legend_name="311 Requests per Neighborhood Council",
)

map.add_child(choro_nc)

# NC tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_nc.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilName', 'councilId', 'count'],
    aliases = ['Neighborhood Council', 'Council ID', '# of Requests'],
    localize = True
))

######

# block-by-block requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
bins = gdf_blk_plt["count"].quantile([0, 0.1, 0.25, 0.5, .75, .95, 1]).tolist()

choro_blk = folium.Choropleth(
    geo_data = gdf_blk_plt.to_json(),
    name="Block Choropleth",
    data = gdf_blk_plt,
    columns=['GEOID20', 'count'],
    key_on="feature.properties.GEOID20",
    bins = bins,
    fill_color="BuPu",
    fill_opacity=0.7,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    legend_name="311 Requests per Block",
)

map.add_child(choro_blk)

# block tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['name', 'councilId', 'GEOID20', 'count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block Census ID', '# of Requests'],
    localize = True
))

# NC boundary lines
folium.GeoJson(data = gdf_nc["geometry"], name = 'NC Boundaries',
               style_function = lambda x:{
                   "color": "#000000",
                   "weight": 1,
                   "opacity": .25,
                   "fill": False
               }
              ).add_to(map)

folium.LayerControl(collapsed = False).add_to(map)

map

In [ ]:
map.save('../streamlit/nc_blk_layered.html')